In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
cd "/content/drive/MyDrive/BTP_AMRITAMAM/"

/content/drive/.shortcut-targets-by-id/1HAUgj9ICrfB46ME_Gce9FK26De_hM_Yv/BTP_AMRITAMAM


In [3]:
#!pip install spectrum
#!pip install git+https://github.com/synergetics/spectrum.git
!pip install hfda
#!pip install -q entropy==v0.1.0 thi is deprecated and the new one is below
!pip install antropy
!pip install git+https://github.com/forrestbao/pyeeg.git # ap, sample, svd entropies---> pyeeg
#!pip install eeglib 
!pip install ordpy #permutation entropy feature
!pip install turbustat

  Created wheel for hfda: filename=hfda-0.1.1-py3-none-any.whl size=2159 sha256=cb2271b8a49e1d55d34b4c82ee2878889e2d9f2a17b64d6acec8f943d0dacb0e
  Stored in directory: /root/.cache/pip/wheels/cc/cf/2e/ee8b2442bdfd74fb48c59230ac1ab1d32574e8d41130ded745
Successfully built hfda
     |████████████████████████████████| 49 kB 3.3 MB/s 
  Created wheel for antropy: filename=antropy-0.1.4-py3-none-any.whl size=16158 sha256=e19f5892dbf652a297a83bd80d6c818b0a61e0073be9e17afdc026555e34f254
  Stored in directory: /root/.cache/pip/wheels/d4/cc/6b/28fa7036bdabc6f1f92fd6a108ae259dbb48f107f676d6b491
Successfully built antropy
  Cloning https://github.com/forrestbao/pyeeg.git to /tmp/pip-req-build-88w53xsn
  Running command git clone -q https://github.com/forrestbao/pyeeg.git /tmp/pip-req-build-88w53xsn
  Created wheel for pyeeg: filename=pyeeg-0.4.4-py2.py3-none-any.whl size=28132 sha256=3f9d14813b3b9a65ec76a644565e75298e3c63f808c17982e00e3ebff2c7a4f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-

In [4]:
import csv
import numpy as np
np.warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import os  
import hfda
import math
import statistics
from scipy.stats import kurtosis
#import entropy as ent
#import eeglib
import scipy as sp
import antropy
import pyeeg
import ordpy
import networkx as nx
#import spectrum

In [5]:
#------------------STATISCTICAL FEATURES-------------------

def peakValue(X):
    val = np.amax(abs(np.array(X)))

    return val

def rmsValue(X):
    n = X.shape[0]

    square = np.sum(np.array(X)**2)
    mean = (square / (float)(n))
    root = math.sqrt(mean)
     
    root = root/4       # normalisation
    #print("root mean square value: ", root)
    return root

def absoluteMean(X):
    mean = np.sum(abs(np.array(X)))
    
    mean = mean/3000    #normalisation
    #print("Absolute mean: ", float(mean))
    return mean

def totalVariation(X):    
    val = np.sum(abs(np.diff(X)))

    val = val/1500    #normalisation
    #print("total variation: ", float(val))
    return val

def hurst(X):
    X = np.array(X)
    N = X.size
    T = np.arange(1, N + 1)
    Y = np.cumsum(X)
    Ave_T = Y / T

    S_T = np.zeros(N)
    R_T = np.zeros(N)

    for i in range(N):
        S_T[i] = np.std(X[:i + 1])
        X_T = Y - T * Ave_T[i]
        R_T[i] = np.ptp(X_T[:i + 1])

    R_S = R_T / S_T
    R_S = np.log(R_S)[1:]
    n = np.log(T)[1:]
    A = np.column_stack((n, np.ones(n.size)))
    [m, c] = np.linalg.lstsq(A, R_S, rcond = None)[0]
    H = m
    return H


def dfa(X, Ave=None, L=None):     # pyeeg official implementation
    X = pd.DataFrame(X).to_numpy()
    X = X.flatten()
    if Ave is None:
        Ave = np.mean(X)

    Y = np.cumsum(X)
    Y -= Ave

    if L is None:
        L = np.floor(len(X) * 1 / (
            2 ** np.array(list(range(4, int(np.log2(len(X))) - 4))))
        )

    F = np.zeros(len(L))  # F(n) of different given box length n

    for i in range(0, len(L)):
        n = int(L[i])                        # for each box length L[i]
        for j in range(0, len(X), n):  # for each box
            if j + n < len(X):
                c = list(range(j, j + n))
                # coordinates of time in the box
                c = np.vstack([c, np.ones(n)]).T
                # the value of data in the box
                y = Y[j:j + n]
                # add residue in this box
                F[i] += np.linalg.lstsq(c, y)[1]
        F[i] /= ((len(X) / n) * n)
    F = np.sqrt(F)

    Alpha = np.linalg.lstsq(np.vstack(
        [np.log(L), np.ones(len(L))]
    ).T, np.log(F))[0][0]

    #print("Detrended Fluctuation Analysis (dfa): ", Alpha)
    return Alpha


def hjorth(X, D=None):    # pyeeg official implementation
    X = pd.DataFrame(X).to_numpy()
    X = X.flatten()
    if D is None:
        D = np.diff(X)
        D = D.tolist()

    D.insert(0, X[0])  # pad the first difference
    D = np.array(D)

    n = len(X)

    M2 = float(sum(D ** 2)) / n
    TP = sum(np.array(X) ** 2)
    M4 = np.sum(np.diff(D)**2)
    M4 = M4 / n

    complexity = np.sqrt(float(M4) * TP / M2 / M2)
    complexity = (complexity - 35) / 10          # normalisation
    mobility = np.sqrt(M2 / TP)
    mobility = mobility*10    # normalisation
    return mobility, complexity  # Hjorth Mobility and Complexity


# ---------------FRACTAL FEATURES---------------------------

def HiguchiFractal(X, k_max):
    D = hfda.measure(X, k_max)
    return D-1    # returning (D-1) instead of returning real answer, i.e., D because for normalisation I want all values in range(0,1)

def KatzFractal(X):
    d = -100000
    n = X.shape[0]
    L = 0.000000
    for i in range(0,n-2):
      tmp1 = (X.loc[i+1]-X.loc[i])*(X.loc[i+1]-X.loc[i]) + 0.4*0.4;
      tmp1 = math.sqrt(tmp1)
      L += tmp1
      tmp2 = math.sqrt((X.loc[i+1]-X.loc[0])*(X.loc[i+1]-X.loc[0]) + (0.4*i)*(0.4*i))    
      d = max(d,tmp2)
    
    a = math.log(n)
    D = a/(a+math.log(d/L))

    #print("Katz Fractal ", D-1);
    return D-1

def PetrosianFractal(X):
    mean = 0.000000
    n = X.shape[0]
    mean = np.sum(np.array(X))

    mean = mean/n

    z = [[0] * 1] * n   # n rows, 1 column, all equal to zero
    #func = lambda x : 1 if (x > mean) else -1
    #z = func(np.array(X))
    for i in range(0,n):
        if (X.loc[i] > mean).bool():
          z[i] = 1
        else:
          z[i] = -1

    nDel = np.sum(abs(np.diff(z)))

    nDel = (1.00*nDel)/2
    a = math.log10(n)
    D = a/(a + math.log10(n/(n+0.4*nDel)))

    #print("Petrosian Fractal ", D-1)
    return D-1

def SevcikFractal(X):
    n = X.shape[0]
    x_min = 100000000
    y_min = 0
    x_max = -10000000000
    y_max = 0.4*(n-1)

    xStar = [0]*n
    yStar = [0]*n

    X1 = np.array(X)
    x_max = np.amax(X1)
    x_min = np.amin(X1)

    xStar = (X1 - x_min) / (x_max - x_min)
    yStar = np.linspace(0, (0.4*(n-1) -y_min)/(y_max-y_min), n)
    xStar = (np.diff(xStar, axis = 0)**2).reshape(xStar.size-1,)
    yStar = np.diff(yStar)**2

    L = np.sum(np.sqrt(xStar + yStar))

    D = 1 + (math.log(L) + math.log(2))/math.log(2*(n-1))
    
    #print("Sevcik Fractal ", D-1)
    return D-1

def findSquareMean(X):
    mean = np.sum(np.array(X)**2)

    #print("mean ", mean)
    return mean

def HajorthParameters(xV):
    n = xV.shape[0];
    dxV = np.diff([xV]);
    ddxV = np.diff([dxV]);
    mx2 = findSquareMean(xV);
    mdx2 = findSquareMean(dxV);
    mddx2 = findSquareMean(ddxV);

    mob = mdx2 / mx2;
    complexity = math.sqrt(mddx2 / mdx2 - mob);
    mobility = math.sqrt(mob);

    #print("Hajorth parameters: mobility = ", mobility, ", complexity = ", complexity);
    return {mobility, complexity};


In [6]:
# Make All Nodes of one csv file

def makeGraphNodes(readFile):
    training = pd.read_csv(readFile)
    #makeFolder(s,saveDirectory)
    n = training.shape[1]
    all_nodes = []

    for node_no in range (0,9):
        labelling = [0] * 17

        i = 0
        # Get Kth Column of Matrix
        x = training.iloc [ : , node_no :node_no+1 ]
        X=np.array(training).T[node_no]
        #print("data ", x)

        # Statistical features
        labelling[0] = kurtosis(x, fisher = True)[0]  #fisher: If True, Fisher’s definition is used (normal ==> 0.0). If False, Pearson’s definition is used (normal ==> 3.0).
        #print("Kurtosis: ", labelling[0])
        #labelling[1] = peakValue(x)
        labelling[1] = rmsValue(x)
        #labelling[3] = (labelling[1]/labelling[2])**2   # Peak-to-Average Power Ratio
        #labelling[4] = (labelling[2]/absoluteMean(x))   # Form factor
        labelling[2] = totalVariation(x)
        labelling[3] = hurst(x)
        #print("Hurst exponent: ", labelling[3])
        labelling[4] = dfa(x)
        labelling[5], labelling[6] = hjorth(x)
        #print("Hjorth mobility, complexity: ", labelling[8], " ", labelling[9])
        labelling[7] = pyeeg.fisher_info(X, 5, 2)
        #print("fisher info", labelling[10]) 

        # Fractal features
        labelling[8] = antropy.higuchi_fd(X);
        labelling[8] = labelling[8] - 1     # normalisation
        #print("higuchi fractal dimension", labelling[11])
        labelling[9] = KatzFractal(x)
        labelling[10] = PetrosianFractal(x)
        labelling[11] = SevcikFractal(x)

        # Entropy features
        s=np.std(X)
        labelling[12] = pyeeg.ap_entropy(X, 2, 0.2*s)
        #Sample Entropy : s(X,m,r) : Generally we take the value of m to be 2 and the value of r to be 0.2*std.
        #the above also applies for ap_entropy  
        #print("\nap entropy: ", labelling[15])
        
        labelling[13] = antropy.sample_entropy(X) 
        #print("simple entropy: ", labelling[16])

        labelling[14] = antropy.svd_entropy(X, 5, 2, normalize=True)
        #print("svd (using entropy library) entropy: ", labelling[17])
        
        labelling[15] = antropy.perm_entropy(X, 5, 2, normalize=True)
        #print("permutation entropy: ", labelling[18])
        
        labelling[16] = antropy.spectral_entropy(X, sf=250, method='welch', normalize=True) # sf is sampling frequency
        #print("spectral entropy: ", labelling[19])

        all_nodes.append(labelling)
       
    
    return all_nodes
    

In [7]:
# Adjacency Matrix representation in Python


class Graph(object):

    # Initialize the matrix
    def __init__(self, size):
        self.adjMatrix = []
        for i in range(size):
            self.adjMatrix.append([0 for i in range(size)])
        self.size = size

    # Add edges
    def add_edge(self, v1, v2, weight):
        self.adjMatrix[v1][v2] = weight
        self.adjMatrix[v2][v1] = weight

    # Remove edges
    def remove_edge(self, v1, v2):
        if self.adjMatrix[v1][v2] == 0:
            print("No edge between %d and %d" % (v1, v2))
            return
        self.adjMatrix[v1][v2] = 0
        self.adjMatrix[v2][v1] = 0

    def __len__(self):
        return self.size

    # Print the matrix
    def print_matrix(self):
        for row in self.adjMatrix:
            for val in row:
                print('{:4}'.format(val), end = " "),
            print("")

def findWeight(v1, v2):
    # find gaussian F(a,b)
    sigma = 100.000

    d = np.sum((np.float_(v1) - np.float_(v2))**2)
    d = math.sqrt(d)
    gaussian = math.exp((-1.0000*d*d)/(2*sigma*sigma))
    #print("gaussian ", round(gaussian,8), end = ", ")

    v1 = np.array(v1)
    v2 = np.array(v2)
    Ta = np.sum(v1**2.00)
    Tb = np.sum(v2**2.00)
    Ta = math.sqrt(Ta)
    Tb = math.sqrt(Tb)
    d2 = math.log(Ta/Tb)
    #print("d ", round(d2,8), end=" , ")

    weight = round(gaussian,8)*round(d2,8)
    if(weight<0.00):
      weight = -1.00*weight

    return round(weight,5)


def mainFunc(all_nodes):
    l = len(all_nodes)
    g = Graph(l)
    for i in range(0,l):
        for j in range(i,l):
            weight = findWeight(all_nodes[i],all_nodes[j])
            g.add_edge(i,j,weight)

    
    A = g.adjMatrix
    #G = nx.from_numpy_matrix(np.matrix(A), create_using=nx.DiGraph)
    
  
    #posi = nx.circular_layout(G)
    #color=['green', 'yellow', 'red', 'blue', 'purple', 'brown', 'cyan', 'grey', 'black'] #'gray', 'olive', 'orange', 'pink', 'gold', 'lightsteelblue', 'peru', 'peachpuff', 'darkseagreen', 'darkkhaki', 'black', 'snow', 'wheat']
    # larger figure size
    #plt.figure(5,figsize=(7,7))
    ###nx.draw(G, posi)
    #nx.draw(G, posi, node_color=color)
    #nx.draw_networkx_edge_labels(G, pos=posi)
    
    #plt.show()

    #g.print_matrix()
    return A



In [ ]:
#code for making adjacency matrix
'''
import csv
import scipy.signal
import numpy as np
import pandas as pd
from skimage import io
import os  
    
def makeFolder(directory,parent_dir):
      # os.makedirs() method will raise  
      # an OSError if the directory to be created already exists  
          
      # Path  
      path = os.path.join(parent_dir, directory)  
          
      # Create the directory  
      os.makedirs(path)  
      print("Directory '% s' created" % directory) 
      return 1


#for all A01-9 folder
for folder in range(8,10):
    #for all A01T_1-4 folder
    for file in range(3,5):
        s = "A0" + str(folder) + "E_" + str(file);   #read file name
        readFolder = "Segmentation_after_channel_selection/A0"+ str(folder)+"/Testing/" + s + "/"   #read file location
        writeFolder = "Graphs/A0"+str(folder)+"/Testing/"  #save location
        #makeFolder(s,writeFolder)
        writeFolder += s + "/"
        dirListing = os.listdir(readFolder)
        print(len(dirListing))
        file_count = len(dirListing)
        for subfileNo in range(1,95):
            readFile = readFolder + str(subfileNo) + ".csv"
            print(readFile)
            #training = pd.read_csv(readFile)
            all_nodes = makeGraphNodes(readFile)
            
            g = mainFunc(all_nodes)

            g2 = g 
            g3 = []
            for row in g2:
              g3.append(",".join(str(item) for item in row))
              
            g4 = "[["
            g4 += "],[".join(str(item) for item in g3) + "]]"
            
            writeFile = writeFolder + str(subfileNo) + ".txt"
            with open(writeFile,'w') as f:
              f.write(g4)
            f.close()
'''

'\nimport csv\nimport scipy.signal\nimport numpy as np\nimport pandas as pd\nfrom skimage import io\nimport os  \n    \ndef makeFolder(directory,parent_dir):\n      # os.makedirs() method will raise  \n      # an OSError if the directory to be created already exists  \n          \n      # Path  \n      path = os.path.join(parent_dir, directory)  \n          \n      # Create the directory  \n      os.makedirs(path)  \n      print("Directory \'% s\' created" % directory) \n      return 1\n\n\n#for all A01-9 folder\nfor folder in range(8,10):\n    #for all A01T_1-4 folder\n    for file in range(3,5):\n        s = "A0" + str(folder) + "E_" + str(file);   #read file name\n        readFolder = "Segmentation_after_channel_selection/A0"+ str(folder)+"/Testing/" + s + "/"   #read file location\n        writeFolder = "Graphs/A0"+str(folder)+"/Testing/"  #save location\n        #makeFolder(s,writeFolder)\n        writeFolder += s + "/"\n        dirListing = os.listdir(readFolder)\n        print(l

In [ ]:

'''
def makeFolder(directory,parent_dir):
      path = os.path.join(parent_dir, directory)  
 
      os.makedirs(path)  
      print("Directory '% s' created" % directory)
      return 1


s = "A01T_1";   #read file name
readFile = "Segmentation/A01/Training/" + s  + "/4.csv"  #read file location
saveDirectory = "Graphs/A01/Training/"  #save location
all_nodes = makeGraphNodes(readFile)
print("################# All Nodes ##############")
print("")
print(all_nodes)


g = mainFunc(all_nodes)
arr = np.array(g)
DF = pd.DataFrame(arr)
csvFile = saveDirectory + s + "/4.csv"
DF.to_csv(r"%s"%csvFile, index = False, header = False)


li2 = [ y for x in g for y in x]
','.join(map(str,li2))
'''

NameError: ignored

In [ ]:
'''
filename = saveDirectory + s + "/trial1.txt"
#g.to_csv(r"%s"%loc,header=False,index=False)
g2 = g 
g3 = []
for row in g2:
  g3.append(",".join(str(item) for item in row))

g4 = "[["
g4 += "],[".join(str(item) for item in g3) + "]]"

print("g2::" , g4)

with open(filename,'w') as f:
  f.write(g4)


  # c3, c4, c8 automatic cchannel selection in EEG signals for classification 2017
'''

NameError: ignored

In [ ]:
#bispectrum features selection
'''
from turbustat.statistics import Bispectrum
from astropy.io import fits
import matplotlib.pyplot as plt

s = "A01T_1";   #read file name
readFile = "Segmentation/A01/Training/" + s  + "/1.csv"  #read file location
saveDirectory = "Graphs/A01/Training/"  #save location
moment0 = pd.read_csv(readFile)

# bispec = Bispectrum(moment0)

spectrum.bispectrumi(moment0[0], nlag=None, nsamp=None, overlap=None, flag='biased', nfft=None, wind=None)


for node_no in range (0,22):
    print("\n", node_no)
    labelling = [0] * 20
'''

In [ ]:
#for genrating the respective directories

import csv
import scipy.signal
import numpy as np
import pandas as pd
from skimage import io
import os 
def makeFolder(directory,parent_dir):
      # os.makedirs() method will raise  
      # an OSError if the directory to be created already exists  
          
      # Path  
      path = os.path.join(parent_dir, directory)  
          
      # Create the directory  
      os.makedirs(path)  
      print("Directory '% s' created" % directory) 
      return 1

for folder in range(2,4):
    #for all A01E_1-4 folder
    for file in range(1,5):
        s = "A0" + str(folder) + "E_" + str(file);   #read file name
        writeFolder = "GraphsNodes/A0"+str(folder)+"/Testing/"  #save location
        makeFolder(s,writeFolder)

Directory 'A02E_1' created
Directory 'A02E_2' created
Directory 'A02E_3' created
Directory 'A02E_4' created
Directory 'A03E_1' created
Directory 'A03E_2' created
Directory 'A03E_3' created
Directory 'A03E_4' created


In [8]:
import csv
import scipy.signal
import numpy as np
import pandas as pd
from skimage import io
import os 

def makeFolder(directory,parent_dir):
      # os.makedirs() method will raise  
      # an OSError if the directory to be created already exists  
          
      # Path  
      path = os.path.join(parent_dir, directory)  
          
      # Create the directory  
      os.makedirs(path)  
      print("Directory '% s' created" % directory) 
      return 1


#for all A01-9 folder
#currently running for folder 1 ONLY 
for folder in range(2,3):
    #for all A01T_1-4 folder
    for file in range(4,5):
        s = "A0" + str(folder) + "E_" + str(file);   #read file name
        readFolder = "Segmentation_after_channel_selection/A0"+ str(folder)+"/Testing/" + s + "/"   #read file location
        writeFolder = "GraphsNodes/A0"+str(folder)+"/Testing/"  #save location
        #makeFolder(s,writeFolder)
        writeFolder += s + "/"
        dirListing = os.listdir(readFolder)
        print(len(dirListing))
        file_count = len(dirListing)
        for subfileNo in range(1,file_count+1):
            readFile = readFolder + str(subfileNo) + ".csv"
            print(readFile)
            #training = pd.read_csv(readFile)
            all_nodes = makeGraphNodes(readFile)
            
            #g = mainFunc(all_nodes)

            g2 = all_nodes 
            g3 = []
            for row in g2:
              g3.append(",".join(str(item) for item in row))
              
            g4 = "[["
            g4 += "],[".join(str(item) for item in g3) + "]]"
            
            writeFile = writeFolder + str(subfileNo) + ".txt"
            with open(writeFile,'w') as f:
              f.write(g4)
            f.close()

145
Segmentation_after_channel_selection/A02/Testing/A02E_4/1.csv
Segmentation_after_channel_selection/A02/Testing/A02E_4/2.csv
Segmentation_after_channel_selection/A02/Testing/A02E_4/3.csv
Segmentation_after_channel_selection/A02/Testing/A02E_4/4.csv
Segmentation_after_channel_selection/A02/Testing/A02E_4/5.csv
Segmentation_after_channel_selection/A02/Testing/A02E_4/6.csv
Segmentation_after_channel_selection/A02/Testing/A02E_4/7.csv
Segmentation_after_channel_selection/A02/Testing/A02E_4/8.csv
Segmentation_after_channel_selection/A02/Testing/A02E_4/9.csv
Segmentation_after_channel_selection/A02/Testing/A02E_4/10.csv
Segmentation_after_channel_selection/A02/Testing/A02E_4/11.csv
Segmentation_after_channel_selection/A02/Testing/A02E_4/12.csv
Segmentation_after_channel_selection/A02/Testing/A02E_4/13.csv
Segmentation_after_channel_selection/A02/Testing/A02E_4/14.csv
Segmentation_after_channel_selection/A02/Testing/A02E_4/15.csv
Segmentation_after_channel_selection/A02/Testing/A02E_4/16.c